In [1]:
import os
import numpy as np
from mido import MidiFile, MidiTrack, Message

global soundfont
global midi_path
global preprocess_audio
global soundfont_path
global gains_path

soundfont = "Yamaha_S6"
midi_name = "velocity_test_69"

soundfont_path = os.path.join("SOURCES/SOUNDFONTS/", soundfont + ".sf2") #Ensure Soundfont is in /SOURCES/SOUNDFONTS/[insert Soundfont name].sf2
source_midi_path = f"3_GENERATE_OUT/MIDI/{midi_name}_{soundfont}_pre.mid"
coefficients_path = os.path.join("SOURCES/SF_COEFFICIENTS/", f"{soundfont}_coefficients.txt")


outputs_path = "4_POSTPROCESS_OUT/"
midi_path = os.path.join(outputs_path, f"MIDI/{midi_name}/")
os.makedirs(midi_path, exist_ok=True)

postprocess_midi =  os.path.join(midi_path, f"{midi_name}_{soundfont}_pre_post.mid")

def logistic_inverse(y, L, k, x0, b):
    try:
        inner = (L / (y - b)) - 1
        if inner <= 0:
            return np.nan
        result = y*(x0 - (1 / k) * np.log(inner))
        # Force the result to be within the 1-127 range
        return int(np.clip(result, 1, 127))
    except Exception:
        return np.nan

def read_logistic(coefficients_path, note_number):
    try:
        with open(coefficients_path, "r") as infile:
            lines = infile.readlines()
    except FileNotFoundError:
        print(f"Error: File not found: {coefficients_path}")
        return None
    except Exception as e:
        print(f"An error occurred while reading the file: {e}")
        return None

    for line in lines:
        line = line.strip()
        if not line:
            continue
        values = line.split()
        if not values:
            continue
        try:
            if int(values[0]) == note_number:
                try:
                    return [float(v) for v in values[1:]]
                except ValueError:
                    print(f"Error: Invalid coefficient format in line: {line}")
                    return None  # Return None if coefficients are not floats
            # No need to convert the first value to int here, just compare
        except ValueError:
            print(f"Skipping line due to non-integer note number: {line}")
            # No return here, continue to the next line

    print(f"Note number {note_number} not found in file {coefficients_path}")
    return None  # Return None if note number is not found

def modify_velocities_by_note(midi_file_path, coefficients_path):
    try:
        midi = MidiFile(midi_file_path)
    except FileNotFoundError:
        print(f"Error: MIDI file not found: {midi_file_path}")
        return None
    except Exception as e:
        print(f"An error occurred while opening the MIDI file: {e}")
        return None

    modified_midi = MidiFile()
    modified_midi.ticks_per_beat = midi.ticks_per_beat

    for track_index, track in enumerate(midi.tracks):
        new_track = MidiTrack()
        for i, event in enumerate(track):
            if event.type == 'note_on' and event.velocity > 0:
                coeffs = read_logistic(coefficients_path, event.note)
                if coeffs:
                    L, k, x0, b = coeffs
                    try:
                        inv = logistic_inverse(event.velocity, L, k, x0, b)
                        if not np.isnan(inv):
                            new_velocity = int(np.clip(inv, 1, 127))  # Ensure it's between 1–127
                            event = event.copy(velocity=new_velocity)
                        else:
                            print(f"Warning: Logistic inverse NaN for note {event.note}, velocity {event.velocity}")
                    except Exception as e:
                        print(f"Math error for note {event.note}: {e}")
            new_track.append(event)
        modified_midi.tracks.append(new_track)

    return modified_midi


if __name__ == "__main__":
    modified_midi_file = modify_velocities_by_note(source_midi_path, coefficients_path)
    if modified_midi_file:
        modified_midi_file.save(postprocess_midi)
        print(f"Modified MIDI file saved to: {postprocess_midi}")



Error: MIDI file not found: 3_GENERATE_OUT/MIDI/velocity_test_69_Yamaha_S6_pre.mid
